In [2]:
import pandas as pd
import os
import pickle
import re

In [3]:
# Load data
DATA_PATH = './data/'
pickle_file = os.path.join(DATA_PATH, 'first_data_batch.pkl')
df = pd.read_pickle(pickle_file)

In [4]:
df.head(5)

,book_id,type,unit,headline,from_date,to_date,additional_info,additional_info_2,combined_info,combined
0,241083,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"לעם ישראל, נאום לרגל ראש-השנה, מלחמה, נצחון, כ...",24/09/1948,24/09/1948,,,,"Headline: לעם ישראל, נאום לרגל ראש-השנה, מלחמה..."
1,241089,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"אדוני ראש הוועד, מוריי ורבותיי, נאום בפני וועד...",03/01/1949,03/01/1949,,,,"Headline: אדוני ראש הוועד, מוריי ורבותיי, נאום..."
2,241095,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"נאום לרגל יום העצמאות, חגים לאומיים, משה רבנו,...",03/05/1949,03/05/1949,,,,"Headline: נאום לרגל יום העצמאות, חגים לאומיים,..."
3,241097,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"לעתידה של הקרן הקיימת לישראל, קרקעות לאום, חכי...",15/05/1948,31/12/1950,,,,"Headline: לעתידה של הקרן הקיימת לישראל, קרקעות..."
4,241105,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"מדינת ישראל והתנועה הציונית, פתיחת הוועד הפועל...",05/05/1949,05/05/1949,,,,"Headline: מדינת ישראל והתנועה הציונית, פתיחת ה..."


In [5]:
print(len(df))

4287


In [15]:
cols = ['type', 'unit']
for col in cols:
    print(f"\nColumn: {col}")
    print(df[col].unique())


Column: type
['ארכיון בן גוריון - מסמך']

Column: unit
['נאומים ומאמרים']


In [51]:
# empty information removed
speeches = df.loc[(df["combined_info"] != "")][['book_id', 'headline', 'combined_info']]
speeches[:5]

,book_id,headline,combined_info
6,241112,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49\n\n\n\nא..."
9,241170,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle East\n\nBy David Ben-Gur...
16,241180,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-..."
23,241225,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...
68,241459,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחב\n\nאני מודה, שהדברים ..."


In [20]:
print(len(speeches))

2548


In [21]:
# we noticed that the initial version of the RAG retrieves the same document but with different id, so we checked it
dup_by_combiened_info = speeches[speeches.duplicated(subset=['combined_info'])]
print(len(dup_by_combiened_info))

194


In [22]:
# Removes duplicate rows based on the 'combined_info' column — keeps only the first occurrence of each unique text
speeches_nondup = speeches.loc[~speeches.duplicated(subset=['combined_info'])]

In [23]:
# Checks if any duplicates still remain after filtering — if the result is 0, all duplicates were successfully removed
print(len(speeches_nondup[speeches_nondup.duplicated(subset=['combined_info'])]))

0


In [24]:
speeches_nondup.head()

,book_id,headline,combined_info
6,241112,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49\n\n\n\nא..."
9,241170,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle East\n\nBy David Ben-Gur...
16,241180,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-..."
23,241225,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...
68,241459,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחב\n\nאני מודה, שהדברים ..."


In [12]:
print(len(speeches_nondup))

2354


In [ ]:
# Pre-processing
def remove_html_tags(text):
    """Remove HTML tags."""
    text = re.sub(r'<[^<]+?>', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    return text

def remove_special_characters(text):
    text = re.sub(r'[^A-Za-z0-9\s\.,;:\?\'\"\!\-\u0590-\u05FF]', '', text)
    return text.strip()

def remove_repeated_substrings(text):
    text = re.sub(r'([.?!,;:—"])\1+', r'\1', text)
    return text.strip()

def remove_extra_spaces(text):
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def preprocess_text(text):
    # Remove HTML tags
    text = remove_html_tags(text)
    
    # Remove special characters
    text = remove_special_characters(text)

    # Remove repeated substrings like dots
    text = remove_repeated_substrings(text)

    # Remove extra spaces between lines and within lines
    text = remove_extra_spaces(text)

    return text.strip()

In [27]:
# Example
example_text = "שלום\nעולם\t<b>test</b>\x0A"
preprocess_text(example_text)


'שלום עולם test'

In [28]:
# Apply preprocessing
speeches_nondup.loc[:, 'combined_info_pp'] = speeches_nondup['combined_info'].apply(preprocess_text)

/tmp/ipykernel_12255/2245366030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speeches_nondup.loc[:, 'combined_info_pp'] = speeches_nondup['combined_info'].apply(preprocess_text)


In [29]:
speeches_nondup.head()

,book_id,headline,combined_info,combined_info_pp
6,241112,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49\n\n\n\nא...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49 אין עמים רב..."
9,241170,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle East\n\nBy David Ben-Gur...,For Israel and Middle East By David Ben-Gurion...
16,241180,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-...","ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט- 19..."
23,241225,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...
68,241459,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחב\n\nאני מודה, שהדברים ...","12.12.57 ח""א במטה המורחב אני מודה, שהדברים של ..."


In [43]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
# detext the langauge of each document
def detect_lang_ld(text):
    try:
        return detect(text)
    except:
        return "unknown"

speeches_nondup['lang'] = speeches_nondup['combined_info_pp'].apply(lambda x: detect_lang_ld(str(x)) if pd.notna(x) else "unknown")


In [ ]:
print(speeches_nondup['lang'].unique())

['he' 'en' 'fr']


In [45]:
print(speeches_nondup['lang'].value_counts())

lang
he    2066
en     285
fr       3
Name: count, dtype: int64


In [47]:
speeches_nondup = speeches_nondup.reset_index(drop=True)

In [48]:
speeches_nondup.head()

,book_id,headline,combined_info,combined_info_pp,lang
0,241112,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49\n\n\n\nא...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49 אין עמים רב...",he
1,241170,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle East\n\nBy David Ben-Gur...,For Israel and Middle East By David Ben-Gurion...,en
2,241180,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-...","ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט- 19...",he
3,241225,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,en
4,241459,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחב\n\nאני מודה, שהדברים ...","12.12.57 ח""א במטה המורחב אני מודה, שהדברים של ...",he


In [49]:
new_data_path = os.path.join(DATA_PATH, "prepd_data.pkl")
with open(new_data_path, "wb") as file: 
    pickle.dump(speeches_nondup, file)